# 1. Imports

In [2]:
import copy
import numba
import matplotlib.pyplot as plt
plt.style.use('classic')
import numpy as np
import pandas as pd
import seaborn as sns 
import statistics
%matplotlib inline 
from os import getcwd
import plotly.express as px
import matplotlib.cm as cm
import pylab as pl

from scipy import sparse
from scipy import stats
from scipy import spatial

import sklearn
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics.pairwise import cosine_similarity

from surprise import accuracy
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise.prediction_algorithms.knns import KNNBasic

getcwd()

'C:\\Users\\tessa\\UVT Thesis\\Untitled Folder'

# 2. Preprocessing

In [3]:
data_inner = pd.read_csv('data_inner_preprocessed.csv')
testdata = pd.read_csv('testdata_inner.csv')

# 3. Clustering

In [4]:
dataset_for_clustering = data_inner[['energy', 'danceability', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                                        'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms', 'time_signature',
                                        'chorus_hit', 'sections']]

# make the clusters
km = KMeans(n_clusters=4, init='k-means++', random_state=42)
song_cluster_pipeline = Pipeline([('scaler', StandardScaler()), 
                                  ('kmeans', km)],verbose=True)

song_cluster_pipeline.fit(dataset_for_clustering)
song_cluster_labels = song_cluster_pipeline.predict(dataset_for_clustering)
data_inner['cluster'] = song_cluster_labels

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.5s
[Pipeline] ............ (step 2 of 2) Processing kmeans, total=  16.3s


In [5]:
cluster0 = data_inner.loc[data_inner['cluster'] == 0]
print(cluster0.shape)

cluster1 = data_inner.loc[data_inner['cluster'] == 1]
print(cluster1.shape)

cluster2 = data_inner.loc[data_inner['cluster'] == 2]
print(cluster2.shape)

cluster3 = data_inner.loc[data_inner['cluster'] == 3]
print(cluster3.shape)

(253288, 30)
(79541, 30)
(66324, 30)
(251873, 30)


In [6]:
testdata_for_clustering = testdata[['energy', 'danceability', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 
                                        'instrumentalness', 'liveness', 'valence', 'tempo','duration_ms', 'time_signature',
                                        'chorus_hit', 'sections']]

test_song_cluster_labels = song_cluster_pipeline.predict(testdata_for_clustering)

# assign cluster to testdata
testdata['cluster'] = test_song_cluster_labels

In [7]:
# split test clusters
# test clusters
testcluster0 = testdata.loc[testdata['cluster'] == 0]
print(testcluster0.shape)

testcluster1 = testdata.loc[testdata['cluster'] == 1]
print(testcluster1.shape)

testcluster2 = testdata.loc[testdata['cluster'] == 2]
print(testcluster2.shape)

testcluster3 = testdata.loc[testdata['cluster'] == 3]
print(testcluster3.shape)

(63595, 30)
(19632, 30)
(16664, 30)
(63361, 30)


# Define the goodness of clusters

In [8]:
# Calinski Harabasz score
from sklearn import metrics
from sklearn.metrics import pairwise_distances

labels_train = data_inner['cluster']
X_train = dataset_for_clustering
labels_test = testdata['cluster']
X_test = testdata_for_clustering

print(metrics.calinski_harabasz_score(X_train, labels_train))
print(metrics.calinski_harabasz_score(X_test, labels_test))

# train: 59314.196498223806
# test: 14759.423834839383
# the higher the better

59314.196498223806
14759.423834839383


In [9]:
# Davies Bouldin score
from sklearn.metrics import davies_bouldin_score

print(davies_bouldin_score(X_train, labels_train))
print(davies_bouldin_score(X_test, labels_test))

# train: 2.6924136877533233
# test: 2.701421278305227
# close to 0 is good

2.692413687753321
2.7014212783052294


In [10]:
# find how similar clusters are: centroids
data_inner.groupby("cluster").mean()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,time_signature,chorus_hit,sections,target,play_count,year,tot_play_count,number_of_obs,mean_play_count,rel_rating
cluster,,,,,,,,,,,,,,,,,,,,,
0,0.500971,0.777231,0.344240,0.839503,0.873058,0.074904,0.085436,0.023783,0.219951,0.472339,...,0.737481,0.232458,0.280991,0.863716,3.050859,1798.310437,182.405211,63.392131,3.150601,0.977882
1,0.460792,0.364443,0.475313,0.625916,0.856854,0.026226,0.655688,0.105956,0.155179,0.370113,...,0.687491,0.218053,0.264643,0.733911,2.931507,1744.688123,176.073145,63.833306,3.008122,0.984539
2,0.475660,0.675722,0.532795,0.734423,0.552636,0.047213,0.192299,0.225132,0.164152,0.404062,...,0.732073,0.223088,0.458285,0.627360,2.996939,1823.148182,181.091234,65.983822,2.998522,0.996124
3,0.709072,0.705550,0.593164,0.803857,0.500482,0.083944,0.144185,0.020068,0.137265,0.680169,...,0.749632,0.229501,0.275656,0.938112,2.898663,1814.334069,181.522128,64.095985,3.096346,0.955379


# 4. Weights

In [11]:
weight_c0 = cluster0.shape[0] / data_inner.shape[0]
weight_c1 = cluster1.shape[0] / data_inner.shape[0]
weight_c2 = cluster2.shape[0] / data_inner.shape[0]
weight_c3 = cluster3.shape[0] / data_inner.shape[0]

weight_tc0 = testcluster0.shape[0] / testdata.shape[0]
weight_tc1 = testcluster1.shape[0] / testdata.shape[0]
weight_tc2 = testcluster2.shape[0] / testdata.shape[0]
weight_tc3 = testcluster3.shape[0] / testdata.shape[0]

# 5. KNN

In [12]:
sim_options = {'name': 'cosine',
              'user_based': False}

model_knn = KNNBasic(sim_options=sim_options, k=18, verbose=True)

In [13]:
reader = Reader(line_format = 'user item rating', rating_scale=(0, 35))
cluster0 = Dataset.load_from_df(cluster0[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster1 = Dataset.load_from_df(cluster1[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster2 = Dataset.load_from_df(cluster2[['user_id', 'song_id', 'rel_rating']], reader = reader)
cluster3 = Dataset.load_from_df(cluster3[['user_id', 'song_id', 'rel_rating']], reader = reader)

print(type(cluster0))

<class 'surprise.dataset.DatasetAutoFolds'>


In [14]:
from surprise.model_selection import train_test_split

train_cluster0, val_cluster0 = train_test_split(cluster0, test_size=0.25, random_state=42)
train_cluster1, val_cluster1 = train_test_split(cluster1, test_size=0.25, random_state=42)
train_cluster2, val_cluster2 = train_test_split(cluster2, test_size=0.25, random_state=42)
train_cluster3, val_cluster3 = train_test_split(cluster3, test_size=0.25, random_state=42)

print(type(train_cluster0), type(val_cluster0))

<class 'surprise.trainset.Trainset'> <class 'list'>


In [15]:
# get train set in a different format
exc00, newtrain_cluster0 = train_test_split(cluster0, test_size=0.999, random_state=42)
exc11, newtrain_cluster1 = train_test_split(cluster1, test_size=0.999, random_state=42)
exc22, newtrain_cluster2 = train_test_split(cluster2, test_size=0.999, random_state=42)
exc22, newtrain_cluster3 = train_test_split(cluster3, test_size=0.999, random_state=42)

print(type(exc00), type(train_cluster0))

<class 'surprise.trainset.Trainset'> <class 'surprise.trainset.Trainset'>


In [16]:
reader = Reader(line_format = 'user item rating', rating_scale=(0, 35))
test_cluster0 = Dataset.load_from_df(testcluster0[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster1 = Dataset.load_from_df(testcluster1[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster2 = Dataset.load_from_df(testcluster2[['user_id', 'song_id', 'rel_rating']], reader = reader)
test_cluster3 = Dataset.load_from_df(testcluster3[['user_id', 'song_id', 'rel_rating']], reader = reader)

print(type(test_cluster0))

<class 'surprise.dataset.DatasetAutoFolds'>


In [17]:
# split data into test and excess dataset to get a different type
#from surprise.model_selection import train_test_split

exc0, test_cluster0 = train_test_split(test_cluster0, test_size=0.999, random_state=42)
exc1, test_cluster1 = train_test_split(test_cluster1, test_size=0.999, random_state=42)
exc2, test_cluster2 = train_test_split(test_cluster2, test_size=0.999, random_state=42)
exc3, test_cluster3 = train_test_split(test_cluster3, test_size=0.999, random_state=42)
print(type(exc0), type(test_cluster0))

<class 'surprise.trainset.Trainset'> <class 'list'>


In [18]:
# find the train mae
prediction0_0 = model_knn.fit(train_cluster0).test(newtrain_cluster0)
prediction1_0 = model_knn.fit(train_cluster1).test(newtrain_cluster1)
prediction2_0 = model_knn.fit(train_cluster2).test(newtrain_cluster2)
prediction3_0 = model_knn.fit(train_cluster3).test(newtrain_cluster3)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [19]:
# find the validation set mae
prediction0_1 = model_knn.fit(train_cluster0).test(val_cluster0)
prediction1_1 = model_knn.fit(train_cluster1).test(val_cluster1)
prediction2_1 = model_knn.fit(train_cluster2).test(val_cluster2)
prediction3_1 = model_knn.fit(train_cluster3).test(val_cluster3)

# check mae for test set for model comparison
prediction0_2 = model_knn.fit(train_cluster0).test(test_cluster0)
prediction1_2 = model_knn.fit(train_cluster1).test(test_cluster1)
prediction2_2 = model_knn.fit(train_cluster2).test(test_cluster2)
prediction3_2 = model_knn.fit(train_cluster3).test(test_cluster3)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [20]:
## average prediction frame

In [22]:
# VALIDATION DATA
result0 = pd.DataFrame(prediction0_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result0.drop(columns = {'details'}, inplace=True)
result0['error'] = abs(result0['base_event'] - result0['predict_event'])
result0.head()

result1 = pd.DataFrame(prediction1_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result1.drop(columns = {'details'}, inplace=True)
result1['error'] = abs(result1['base_event'] - result1['predict_event'])
result1.head()

result2 = pd.DataFrame(prediction2_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result2.drop(columns = {'details'}, inplace=True)
result2['error'] = abs(result2['base_event'] - result2['predict_event'])
result2.head()

result3 = pd.DataFrame(prediction3_1, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result3.drop(columns = {'details'}, inplace=True)
result3['error'] = abs(result3['base_event'] - result3['predict_event'])
result3.head()

,visitor_id,item_id,base_event,predict_event,error
0,f4543bfafbaed53bd9257735d206f0fc2aa175d1,SOUSZSW12AB01800C2,0.304762,1.549799,1.245037
1,0a156afa7128a591fdd16947deb52cc6c4f0b6b4,SOXDMTH12A58A80709,2.496970,0.998851,1.498119
2,5df8828a9a120adeb374aeb9fc82922ba4ab00e1,SOLRGVL12A8C143BC3,0.342495,0.590408,0.247913
3,5462543e74bc8f45c9d3effa7998bbacfeb02e63,SOGZHXG12AF72A0EE8,0.480932,0.897091,0.416159
4,77e0792d2bfa3153c61a73c4ab90ab89ae810324,SOLRGVL12A8C143BC3,0.761194,0.578466,0.182728


In [23]:
# TRAIN DATA
result0 = pd.DataFrame(prediction0_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result0.drop(columns = {'details'}, inplace=True)
result0['error'] = abs(result0['base_event'] - result0['predict_event'])
result0.head()

result1 = pd.DataFrame(prediction1_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result1.drop(columns = {'details'}, inplace=True)
result1['error'] = abs(result1['base_event'] - result1['predict_event'])
result1.head()

result2 = pd.DataFrame(prediction2_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result2.drop(columns = {'details'}, inplace=True)
result2['error'] = abs(result2['base_event'] - result2['predict_event'])
result2.head()

result3 = pd.DataFrame(prediction3_0, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result3.drop(columns = {'details'}, inplace=True)
result3['error'] = abs(result3['base_event'] - result3['predict_event'])
result3.head()

,visitor_id,item_id,base_event,predict_event,error
0,b55937bc8524fefefbd46f08602ad5ce6b434a77,SOKUTUM12A6701D9CD,1.125000,1.125000,2.220446e-16
1,3b19a717778a00d14e3382d134a4a7f6e668580c,SORWGRQ12A8C13FFA1,0.733333,0.733333,0.000000e+00
2,f87b278ef1bcc4f84cd277d8daf3a7eb95554b58,SOPCMUQ12A67ADA1C3,0.761905,1.410293,6.483880e-01
3,48a7e5ef86a7af95e2b6d83ca3c742a23c377288,SOIBSWV12A6D4F6AB3,2.158273,1.591931,5.663424e-01
4,69b5cc43b69f1d294ea0437d16a973d975e90431,SOLVMXY12A8C131E36,0.839080,0.839080,1.110223e-16


In [37]:
# TEST DATA
result0 = pd.DataFrame(prediction0_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result0.drop(columns = {'details'}, inplace=True)
result0['error'] = abs(result0['base_event'] - result0['predict_event'])
result0.head()

result1 = pd.DataFrame(prediction1_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result1.drop(columns = {'details'}, inplace=True)
result1['error'] = abs(result1['base_event'] - result1['predict_event'])
result1.head()

result2 = pd.DataFrame(prediction2_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result2.drop(columns = {'details'}, inplace=True)
result2['error'] = abs(result2['base_event'] - result2['predict_event'])
result2.head()

result3 = pd.DataFrame(prediction3_2, columns=['visitor_id', 'item_id', 'base_event', 'predict_event', 'details'])
result3.drop(columns = {'details'}, inplace=True)
result3['error'] = abs(result3['base_event'] - result3['predict_event'])
result3.head()

,visitor_id,item_id,base_event,predict_event,error
0,042f39f8945a600b7bef1aee3b0905c251d4d933,SOPCEOB12AF72A778A,0.427692,0.906908,0.479216
1,033a83baff9dc3bd98275c81c50514e330ba38c5,SOSIPFF12AB01853C6,0.990826,1.561087,0.570261
2,e695b87b97d3f78131cd56ee694dca8971ef2a39,SONLDNL12A8C13B9EE,0.455268,1.017305,0.562037
3,fd3abf5e4e9908f577eaa8347d6d981e80f78689,SOJSTYO12A8C13F200,0.358586,0.829874,0.471288
4,eef2088ee43403dfda3f3cfd498cfa982ba8e40a,SOGSAYQ12AB018BA14,0.794521,0.953497,0.158976


In [38]:
# do not merge

## result0
play_count_categories = []

for number in result0['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result0 = result0.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result0['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result0 = result0.assign(predict_binary = play_count_categories1)

result0.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,5728516deb82e98f11dec23ef975eb3ccbb79eae,SOGTQNI12AB0184A5C,0.434043,0.881289,0.447246,0,0
1,464db966b212bc1f3c35ca6e78d4380233d38ab0,SOLARJV12AB018306B,0.275862,0.275862,0.000000,0,0
2,b97a9186cc2a2669d761bc543c9810c9e08bfc03,SOERYLG12A6701F07F,4.218750,1.784534,2.434216,1,1
3,196b6ef7f43b53978293fbc8812e08ed108b82c1,SOBONKR12A58A7A7E0,5.739130,5.739130,0.000000,1,1
4,5051ed1c1798559ed7dc1331be64d5eaee1b084e,SONTIBS12A6D4F9CFE,0.530792,1.075307,0.544516,0,1


In [39]:
# do not merge

## result1
play_count_categories = []

for number in result1['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
        
result1 = result1.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result1['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result1 = result1.assign(predict_binary = play_count_categories1)

result1.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,c4a5b793ce5f2d36e759849e66df74917af1df3e,SOYTEUF12A8C1438E1,0.410256,0.410256,0.000000,0,0
1,c7d417f05283d34f02abc5be132a907da8249c4a,SOESTZQ12A67021556,3.880435,2.470335,1.410100,1,1
2,06fe31b2859b658aae821ffb5b78d29ddfd4e016,SOPKQGW12AAF3B17AB,0.671053,0.813904,0.142851,0,0
3,7bda2ec5d80264762cc8052aec6300f878664309,SOTVLQY12A58A798C2,1.582090,1.065049,0.517040,1,1
4,c31dfc9418e32ced5e5abc0d6611ee1ae09c1d08,SOSLZXV12A8C1354C9,0.554455,0.940880,0.386424,0,0


In [40]:
# do not merge

## result2
play_count_categories = []

for number in result2['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result2 = result2.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result2['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result2 = result2.assign(predict_binary = play_count_categories1)

result2.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,4e0a93ab4c4af026fd5fc4fd61c1e51a8f408940,SOLJWIQ12A6D4FA875,0.750000,0.750000,1.110223e-16,0,0
1,cacd0239e8c02945610245c55aa81a130f1e0dc4,SONTQUO12A6D4F7D8B,1.288136,1.288136,0.000000e+00,1,1
2,442f41ebda212bd4055fdc5ba07c4ee41a3d68d4,SORVKCB12AB0188D90,1.787234,1.787234,0.000000e+00,1,1
3,0b1bc9df8de4bf7b72e2d0fb2e841939701ebb91,SOSJRJP12A6D4F826F,0.360544,0.721088,3.605442e-01,0,0
4,1a0b3811e04c027ee717b836fab092dbaff4ace3,SOUBXSF12A6701D23C,2.743243,1.848188,8.950548e-01,1,1


In [41]:
# do not merge

## result3
play_count_categories = []

for number in result3['base_event']:
    if number < 1:
        play_count_categories.append(0)
    else:
        play_count_categories.append(1)
        
result3 = result3.assign(base_binary = play_count_categories)

play_count_categories1 = []

for number in result3['predict_event']:
    if number < 1:
        play_count_categories1.append(0)
    else:
        play_count_categories1.append(1)
        
result3 = result3.assign(predict_binary = play_count_categories1)

result3.head()

,visitor_id,item_id,base_event,predict_event,error,base_binary,predict_binary
0,042f39f8945a600b7bef1aee3b0905c251d4d933,SOPCEOB12AF72A778A,0.427692,0.906908,0.479216,0,0
1,033a83baff9dc3bd98275c81c50514e330ba38c5,SOSIPFF12AB01853C6,0.990826,1.561087,0.570261,0,1
2,e695b87b97d3f78131cd56ee694dca8971ef2a39,SONLDNL12A8C13B9EE,0.455268,1.017305,0.562037,0,1
3,fd3abf5e4e9908f577eaa8347d6d981e80f78689,SOJSTYO12A8C13F200,0.358586,0.829874,0.471288,0,0
4,eef2088ee43403dfda3f3cfd498cfa982ba8e40a,SOGSAYQ12AB018BA14,0.794521,0.953497,0.158976,0,0


In [42]:
# calculate accuracy
from sklearn.metrics import confusion_matrix

y_actual0 = pd.Series(result0['base_binary'], name='actual')
y_predict0 = pd.Series(result0['predict_binary'], name='predicted')
con_mat0 = (pd.crosstab(y_actual0, y_predict0)) / (pd.crosstab(y_actual0, y_predict0)).sum(axis=1)

y_actual1 = pd.Series(result1['base_binary'], name='actual')
y_predict1 = pd.Series(result1['predict_binary'], name='predicted')
con_mat1 = (pd.crosstab(y_actual1, y_predict1)) / (pd.crosstab(y_actual1, y_predict1)).sum(axis=1)

y_actual2 = pd.Series(result2['base_binary'], name='actual')
y_predict2 = pd.Series(result2['predict_binary'], name='predicted')
con_mat2 = (pd.crosstab(y_actual2, y_predict2)) / (pd.crosstab(y_actual2, y_predict2)).sum(axis=1)

y_actual3 = pd.Series(result3['base_binary'], name='actual')
y_predict3 = pd.Series(result3['predict_binary'], name='predicted')
con_mat3 = (pd.crosstab(y_actual3, y_predict3)) / (pd.crosstab(y_actual3, y_predict3)).sum(axis=1)

In [43]:
print('Result 0:', con_mat0)
print('\n')

print('Result 1:', con_mat1)
print('\n')

print('Result 2:', con_mat2)
print('\n')

print('Result 3:', con_mat3)
print('\n')

Result 0: predicted         0         1
actual                       
0          0.766838  0.529581
1          0.151391  0.656145


Result 1: predicted         0         1
actual                       
0          0.844243  0.359831
1          0.143556  0.668354


Result 2: predicted         0         1
actual                       
0          0.859555  0.318102
1          0.135510  0.693077


Result 3: predicted         0         1
actual                       
0          0.780099  0.530992
1          0.152074  0.632788




In [44]:
TP = (con_mat0[1][1])*weight_tc0 + (con_mat1[1][1])*weight_tc1 + (con_mat2[1][1])*weight_tc2 + (con_mat3[1][1])*weight_tc3# + (con_mat4[1][1])*weight_tc4 + (con_mat5[1][1])*weight_tc5 
TN = (con_mat0[0][0])*weight_tc0 + (con_mat0[0][0])*weight_tc1 + (con_mat0[0][0])*weight_tc2 + (con_mat0[0][0])*weight_tc3# + (con_mat0[0][0])*weight_tc4 + (con_mat0[0][0])*weight_tc5 
FP = (con_mat0[0][1])*weight_tc0 + (con_mat1[0][1])*weight_tc1 + (con_mat2[0][1])*weight_tc2 + (con_mat3[0][1])*weight_tc3# + (con_mat4[0][1])*weight_tc4 + (con_mat5[0][1])*weight_tc5 
FN = (con_mat0[1][0])*weight_tc0 + (con_mat1[1][0])*weight_tc1 + (con_mat2[1][0])*weight_tc2 + (con_mat3[1][0])*weight_tc3# + (con_mat4[1][0])*weight_tc4 + (con_mat5[1][0])*weight_tc5 

print('TP:',TP,'\nTN:',TN,'\nFP:',FP,'\nFN:',FN)

TP: 0.6523181863550851 
TN: 0.7668382036226792 
FP: 0.14909288787624625 
FN: 0.48812858598387204


In [45]:
from sklearn import metrics
recall0 = metrics.recall_score(y_actual0, y_predict0)
recall1 = metrics.recall_score(y_actual1, y_predict1)
recall2 = metrics.recall_score(y_actual2, y_predict2)
recall3 = metrics.recall_score(y_actual3, y_predict3)
recall = recall0*weight_tc0 + recall1*weight_tc1 + recall2*weight_tc2 + recall3*weight_tc3 

precision0 = metrics.precision_score(y_actual0, y_predict0)
precision1 = metrics.precision_score(y_actual1, y_predict1)
precision2 = metrics.precision_score(y_actual2, y_predict2)
precision3 = metrics.precision_score(y_actual3, y_predict3)
precision = precision0*weight_tc0 + precision1*weight_tc1 + precision2*weight_tc2 + precision3*weight_tc3

accuracy0 = metrics.accuracy_score(y_actual0, y_predict0)
accuracy1 = metrics.accuracy_score(y_actual1, y_predict1)
accuracy2 = metrics.accuracy_score(y_actual2, y_predict2)
accuracy3 = metrics.accuracy_score(y_actual3, y_predict3)
accuracy = accuracy0*weight_tc0 + accuracy1*weight_tc1 + accuracy2*weight_tc2 + accuracy3*weight_tc3

fscore0 = metrics.f1_score(y_actual0, y_predict0)
fscore1 = metrics.f1_score(y_actual1, y_predict1)
fscore2 = metrics.f1_score(y_actual2, y_predict2)
fscore3 = metrics.f1_score(y_actual3, y_predict3)
fscore = fscore0*weight_tc0 + fscore1*weight_tc1 + fscore2*weight_tc2 + fscore3*weight_tc3
print('Recall:', round(recall,3), '\nPrecision:', round(precision,3), '\nAccuracy:', round(accuracy,3), '\nF1 score:', 
      round(fscore,3))

Recall: 0.652 
Precision: 0.575 
Accuracy: 0.749 
F1 score: 0.61


In [46]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# NOG VAN APARTE CLUSTERS MAKEN
conmat0 = confusion_matrix(y_actual0, y_predict0)
conmat1 = confusion_matrix(y_actual1, y_predict1)
conmat2 = confusion_matrix(y_actual2, y_predict2)
conmat3 = confusion_matrix(y_actual3, y_predict3)

TP = conmat0[1][1]*weight_tc0 + conmat1[1][1]*weight_tc1 + conmat2[1][1]*weight_tc2 + conmat3[1][1]*weight_tc3
TN = conmat0[0][0]*weight_tc0 + conmat1[0][0]*weight_tc1 + conmat2[0][0]*weight_tc2 + conmat3[0][0]*weight_tc3
FP = conmat0[0][1]*weight_tc0 + conmat1[0][1]*weight_tc1 + conmat2[0][1]*weight_tc2 + conmat3[0][1]*weight_tc3
FN = conmat0[1][0]*weight_tc0 + conmat1[1][0]*weight_tc1 + conmat2[1][0]*weight_tc2 + conmat3[1][0]*weight_tc3

print('TP', TP, '\nTN', TN, '\nFP', FP, '\nFN', FN)
TPshare = TP/(TP + TN + FP + FN)
TNshare = TN/(TP + TN + FP + FN)
FPshare = FP/(TP + TN + FP + FN)
FNshare = FN/(TP + TN + FP + FN)
print('\nTP', TPshare, '\nTN', TNshare, '\nFP', FPshare, '\nFN', FNshare)

TP 10353.618399774583 
TN 29132.22985323304 
FP 8248.727917575283 
FN 5639.390494450297

TP 0.19398255454297217 
TN 0.5458134681288057 
FP 0.15454590379881317 
FN 0.10565807352940888


In [47]:
print(conmat0)
print(conmat1)
print(conmat2)
print(conmat3)

[[33826 10285]
 [ 6678 12743]]
[[11556  2132]
 [ 1965  3960]]
[[9927 1622]
 [1565 3534]]
[[34918  9843]
 [ 6807 11730]]


In [48]:
ttl_TP = conmat0[1][1] + conmat1[1][1] + conmat2[1][1] + conmat3[1][1]
ttl_TN = conmat0[0][0] + conmat1[0][0] + conmat2[0][0] + conmat3[0][0]
ttl_FP = conmat0[0][1] + conmat1[0][1] + conmat2[0][1] + conmat3[0][1]
ttl_FN = conmat0[1][0] + conmat1[1][0] + conmat2[1][0] + conmat3[1][0]

print(ttl_TP, ttl_TN, ttl_FP, ttl_FN)

31967 90227 23882 17015


In [49]:
"""[[33826 10285]
 [ 6678 12743]]
[[11556  2132]
 [ 1965  3960]]
[[9927 1622]
 [1565 3534]]
[[34918  9843]
 [ 6807 11730]]"""

'[[135085  40988]\n [ 26387  50575]]\n[[46776  8637]\n [ 7870 16179]]\n[[39501  6485]\n [ 6280 13992]]\n[[138569  39367]\n [ 27258  46428]]'